In [ ]:
from typing import List

class Course:
    def __init__(self, unit_name: str, unit_code: str, pre_req: List[str], semester: int):
        self.unit_name = unit_name
        self.unit_code = unit_code
        self.pre_req = pre_req
        self.semester = semester

class Student:
    def __init__(self, st_id: str, name: str, gender: str, unit_passed: List[str], semester: int):
        self.st_id = st_id
        self.name = name
        self.gender = gender
        self.unit_passed = unit_passed
        self.semester = semester

    def has_met_prerequisites(self, course: Course) -> bool:
        return all(prereq in self.unit_passed for prereq in course.pre_req)

class CourseEnrollmentSystem:
    def __init__(self, course_structure):
        self.course_structure = course_structure

    def get_eligible_courses(self, student: Student) -> List[str]:
        eligible_courses = [
            course.unit_code
            for course in self.course_structure
            if course.semester == student.semester
            and student.has_met_prerequisites(course)
            and not (course.unit_name == "Women in STEM" and student.gender.lower() != "f")
        ]
        return eligible_courses

    def enroll_students(self, student_enrollment_data: List[dict]) -> List[dict]:
        enrollments = []
        for student_data in student_enrollment_data:
            student = Student(student_data["st_id"], student_data["name"], student_data["gender"],
                              student_data["unit_passed"], student_data["semester"])
            eligible_courses = self.get_eligible_courses(student)
            if len(eligible_courses) >= 2:
                enrollments.append({
                    "enroll_code": f"{student.st_id}-{student.semester}",
                    "unit_codes": eligible_courses,
                    "st_id": student.st_id,
                    "semester": student.semester
                })
        return enrollments

    def recommend_courses(self, student: Student) -> List[str]:
        enrollment_background = student.unit_passed
        recommended_courses = [
            course.unit_code
            for course in self.course_structure
            if student.has_met_prerequisites(course)
            and (course.unit_name != "Women in STEM" or student.gender.lower() == "f")
            and not (course.unit_name == "Women in STEM"
                     and any(course_code in enrollment_background for course_code in course.pre_req)
                     and int(student.st_id[-1]) % 2 != 0)
            and course.unit_code not in enrollment_background
        ]
        return recommended_courses


def get_student_enrollment_data() -> List[dict]:
    student_enrollment_data = []
    num_students = int(input("Enter the number of students: "))
    for _ in range(num_students):
        student_data = {
            "st_id": input("Enter student ID: "),
            "name":  input("Enter student name: "),
            "gender": input("Enter gender: "),
            "unit_passed": input("Enter passed units (comma-separated): ").split(", "),
            "semester": int(input("Enter current semester: "))
        }
        student_enrollment_data.append(student_data)
    return student_enrollment_data

def print_enrollments(enrollments: List[dict]) -> None:
    for enrollment in enrollments:
        print("Enrollment:")
        print(f"Enroll Code: {enrollment['enroll_code']}")
        print(f"Unit Codes: {', '.join(enrollment['unit_codes'])}")
        print(f"Student ID: {enrollment['st_id']}")
        print(f"Semester: {enrollment['semester']}")
        print()

def main():
    print("Welcome to the Course Enrollment System!")    
    course_structure = [
        Course("Coding for BA", "BCO7006", [], 1),
        Course("Biz analytics", "BCO7000", [], 1),
        Course("Predictive Analytics", "BCO6008", ["BCO7006"], 2),
        Course("Machine Learning", "BCO7007", ["BCO7000", "BCO7006"], 2),
        Course("Women in STEM", "WOM1000", [], "elective")
    ]

    enrollment_system = CourseEnrollmentSystem(course_structure)

    student_enrollment_data = get_student_enrollment_data()
    enrollments = enrollment_system.enroll_students(student_enrollment_data)
    print_enrollments(enrollments)

    print("\nEnter your information to get course recommendations:")
    user_student = Student(
        input("Enter your student ID: "),
        input("Enter your name: "),
        input("Enter your gender: "),
        input("Enter passed units (comma-separated): ").split(", "),
        int(input("Enter current semester: "))
    )

    recommended_courses = enrollment_system.recommend_courses(user_student)
    print("\nRecommended courses for", user_student.name, "(", user_student.st_id, "):")
    for course_code in recommended_courses:
        print(course_code)

if __name__ == "__main__":
    main()